In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import plotly.figure_factory as ff
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import datetime as dt

from scipy.spatial import Delaunay
from scipy.stats import linregress
from scipy.optimize import curve_fit
from getdist import plots, MCSamples
from iminuit import Minuit
from iminuit.cost import LeastSquares

from qubic.lib.Qgps import GPSAntenna
import qubic.lib.Calibration.Qfiber as ft
 
%matplotlib inline

In [ ]:
# If True, allow plots for debug
DEBUG = False

# Import data

In [ ]:
### Build GPS data file path
data_path = "calsource_orientation.dat"

### Define the distance between the two antennas
distance_between_antennas = 1.3 # Just to initialize GPSAntenna

gps_antenna = GPSAntenna(data_path, distance_between_antennas)

names = np.array(['North', 'East', 'Down', 'Roll', 'Yaw'])
print(gps_antenna._datetime[8100])

del data_path, distance_between_antennas

In [ ]:
index_ini = gps_antenna.get_observation_indices(gps_antenna._datetime, np.array([dt.datetime(year=2025, month=3, day=14, hour=14, minute=33, second=3)]))[0]
print('Initial index = ', index_ini)
print('size observation times : ', len(gps_antenna._datetime))

In [ ]:
gps_antenna.plot_gps_data_plotly(index_ini, -1, [-10, 10], [-2*np.pi, 2*np.pi])

In [ ]:
gps_antenna._datetime[61501]

In [ ]:
index_stop = gps_antenna.get_observation_indices(gps_antenna._datetime, np.array([dt.datetime(2025, 3, 14, 16, 59, 40)]))[0]
position_limit = [-8, 8]
angle_limit = [-np.pi, np.pi]


gps_antenna.plot_gps_data_plotly(index_start=index_ini, index_stop=index_stop, position_limit=position_limit, angle_limit=angle_limit)

In [ ]:
plt.plot(gps_antenna._datetime, gps_antenna.rpN)
mask = (gps_antenna.rpN<10) & (gps_antenna.rpN>-10) & (gps_antenna.rpE<10) & (gps_antenna.rpE>-10) & (gps_antenna.rpD<10) & (gps_antenna.rpD>-10) & (gps_antenna.roll<2*np.pi) & (gps_antenna.roll>-2*np.pi) & (gps_antenna.yaw<2*np.pi) & (gps_antenna.yaw>-2*np.pi)
plt.figure()
gps_antenna.rpN = gps_antenna.rpN[mask]
gps_antenna.rpE = gps_antenna.rpE[mask]
gps_antenna.rpD = gps_antenna.rpD[mask]
gps_antenna.roll = gps_antenna.roll[mask]
gps_antenna.yaw = gps_antenna.yaw[mask]
gps_antenna._datetime = gps_antenna._datetime[mask]

In [ ]:
gps_antenna.plot_gps_data(index_ini, index_stop)
gps_antenna.plot_gps_data_plotly(index_ini, index_stop)

# Build observation time indices

In [ ]:
# Array containing the starting time of each configuration
array_hours_begins = np.array([14, 14, 14, 14, 14, 15, 15, 15, 15, 15, 15, 15, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16]) #, 8, 8, 8, 8, 8, 8])
array_minutes_begins = np.array([33, 38, 44, 49, 55, 5, 10, 32, 38, 46, 52, 57, 3, 8, 14, 20, 26, 31, 37, 42, 48, 53]) #, 15, 21, 26, 32, 37, 43])
array_seconds_begins = np.array([5, 40, 0, 0, 0, 0, 30, 0, 0, 0, 0, 30, 0, 40, 40, 30, 0, 30, 0, 30, 0, 30]) #, 0, 0, 30, 0, 30, 30])

# Array containing the ending time of each configuration, build by substracting 1 minute to the starting time
# In this case, each time correspond to do the ending time of the previous configuration
array_minutes_ending = array_minutes_begins.copy() - 1
array_minutes_ending[1] = 0

# Array containing the seconds of each configuration
array_seconds_ending = np.zeros(array_hours_begins.shape, dtype=int)

# Build the array of datetime associated with each configuration, to delimit the time when the antenna are moving or not
array_datetime = np.array([])
index_array = np.array([], dtype=int)
for index in range(array_hours_begins.shape[0]):
    array_datetime = np.append(array_datetime, dt.datetime(year=2025, month=3, day=14, hour=array_hours_begins[index], minute=array_minutes_begins[index], second=array_seconds_begins[index]))
for index in range(1, array_hours_begins.shape[0]):
    array_datetime = np.append(array_datetime, dt.datetime(year=2025, month=3, day=14, hour=array_hours_begins[index], minute=array_minutes_ending[index], second=array_seconds_ending[index]))
#array_datetime = np.append(array_datetime, dt.datetime(year=2025, month=3, day=14, hour=11, minute=33, second=0))

for index in range(array_datetime.shape[0]):
    print(index, array_datetime[index])
    index_array = np.append(index_array, gps_antenna.get_observation_indices(gps_antenna._datetime, np.array([array_datetime[index]])))
        
del array_hours_begins, array_minutes_begins, array_minutes_ending, array_seconds_begins, array_seconds_ending, array_datetime, index

## Test

In [ ]:
if DEBUG:
    for index in range(int(index_array.size/2)):
        plt.figure(figsize=(15, 8))
        plt.plot(gps_antenna._datetime[index_ini:], gps_antenna.roll[index_ini:])
        plt.xlabel('Date')
        plt.ylabel('Angle (rad)')
        plt.title('GPS Roll Angle')
        plt.vlines(gps_antenna._datetime[index_array[index]], 0, 2*np.pi, color='r', linestyle='--')
        plt.vlines(gps_antenna._datetime[index_array[index + int(index_array.size/2)]], 0, 2*np.pi, color='r', linestyle='--')
        plt.show()
        
    del index

In [ ]:
# Plot to verify the previous delimitation
fig, ax1 = plt.subplots(figsize = (15,5))

color_a = 'tab:pink'
color_r = 'tab:red'
color_b = 'tab:blue'
color_d = 'tab:green'
color_c = 'tab:brown'

ax1.set_xlabel('Date')
ax1.set_ylabel('Position Vector Components (m)', color = color_r)
ax1.plot(gps_antenna._datetime[index_ini:], gps_antenna.rpN[index_ini:], color = color_r, label = 'North component')
ax1.plot(gps_antenna._datetime[index_ini:], gps_antenna.rpE[index_ini:], color = color_b, label = 'East component')
ax1.plot(gps_antenna._datetime[index_ini:], gps_antenna.rpD[index_ini:], color = color_d, label = 'Up component')

ax2 = ax1.twinx()

ax2.plot(gps_antenna._datetime[index_ini:], gps_antenna.roll[index_ini:], color = color_a, label = 'Roll angle')
ax2.plot(gps_antenna._datetime[index_ini:], gps_antenna.yaw[index_ini:], color = color_c, label = 'Yaw angle')
ax2.set_xlabel('Date')
ax2.set_ylabel('Angles (rad)', color = color_a)

for obs_index in index_array:
    ax1.vlines(gps_antenna._datetime[obs_index], 3.5, -1.5, 'grey', linestyles='--')
ax1.vlines(gps_antenna._datetime[index_array[0]], 3.5, -1.5, 'r', linestyles='--')
ax1.vlines(gps_antenna._datetime[index_array[0 + int(index_array.shape[0]/2)]], 3.5, -1.5, 'r', linestyles='--')

fig.tight_layout()
ax1.set_title("Position vector components")
fig.legend(bbox_to_anchor=(1, 1), loc='upper left')
plt.show()

del fig, ax1, ax2, color_a, color_r, color_b, color_d, color_c, obs_index

In [ ]:
# Function to remove the movement between each configurations
def only_data(array, indices):
    data_array = np.array([])
    
    for i in range(int(indices.shape[0]/2)):
        data_array = np.append(data_array, array[indices[i]:indices[i+int(indices.shape[0]/2)]])
    
    return data_array

In [ ]:
data_rpN = only_data(gps_antenna.rpN, index_array)
data_rpE = only_data(gps_antenna.rpE, index_array)
data_rpD = only_data(gps_antenna.rpD, index_array)
data_roll = only_data(gps_antenna.roll, index_array)
data_yaw = only_data(gps_antenna.yaw, index_array)

In [ ]:
data_rpN.shape

In [ ]:
index_array.shape

In [ ]:
# Plot to verify the filtering of the time when the antennas are moving
fig, ax1 = plt.subplots(figsize = (15,5))

color_a = 'tab:pink'
color_r = 'tab:red'
color_b = 'tab:blue'
color_d = 'tab:green'
color_c = 'tab:brown'

ax1.set_xlabel('Date')
ax1.set_ylabel('Position Vector Components (m)', color = color_r)
ax1.plot(data_rpN, color = color_r, label = 'North component')
ax1.plot(data_rpE, color = color_b, label = 'East component')
ax1.plot(data_rpD, color = color_d, label = 'Up component')

ax2 = ax1.twinx()

ax2.plot(data_roll, color = color_a, label = 'Roll angle')
ax2.plot(data_yaw, color = color_c, label = 'Yaw angle')
ax2.set_xlabel('Date')
ax2.set_ylabel('Angles (rad)', color = color_a)

fig.tight_layout()
ax1.set_title("Position vector components")
fig.legend(bbox_to_anchor=(1, 1), loc='upper left')
plt.show()

del fig, ax1, ax2, color_a, color_r, color_b, color_d, color_c